# **Herramientas de Análisis**

---
---

<!-- script html for image -->

<center>


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/Sw-ep1-logo.svg/1280px-Sw-ep1-logo.svg.png"  width="500" height="150"/>


<br>

Fuente de la imagen: [https://es.wikipedia.org](https://es.wikipedia.org/wiki/Star_Wars:_Episodio_I_-_La_amenaza_fantasma)

</center>

<br>

# 9 - Anexo: Readme de Python

## 9.1. Preparación del entorno Colab

Desde [Colab](http://colab.research.google.com), hay que clonar el repositorio cada vez que inicias un nuevo libro. En los libros se incluye el código necesario para ello.

## 9.2. Preparación Entorno Local-Jupyter (opcional)

#### CLONAR REPOSITORIO

En local, puedes utilizar el mismo proyecto que has clonado en el [README DE R](../R/README.md).

Para actualizarlo de nuevo, desde consola:

```
cd mbdds_fc20
git pull
cd Python
```

#### CREAR EN LOCAL UN NUEVO ENVIRONMENT DE ANACONDA

Abrimos una línea de comandos (con *Anaconda 3.0* ya disponible).

- **Windows**: escribimos `Anaconda` en el menú "Inicio" y aparecerá la consola MS-DOS de Anaconda.
- **Linux**: Abrimos Terminal.

```
conda deactivate
conda create -n mbdds_rpy20 python=3.6.9
conda activate mbdds_rpy20
```

Verifica que se ha creado y está activo:

```
conda info --envs
```

#### INSTALA LAS LIBRERIAS DE PYTHON

```
cd mbdds_fc20/Python
conda activate mbdds_rpy20
python -m pip install -r requirementsColab.txt
```

#### PUBLICAR EL KERNEL

Para acceder al nuevo environment desde Jupyter necesitas publicar el kernel:

```
python -m ipykernel install --user --name mbdds_rpy20 --display-name "mbdds_rpy20"
```

Puede tardar unos minutos en publicarse.

#### LANZAR ENTORNO JUPYTER NOTEBOOK

Para acceder al servidor Jupyter:

```
conda activate mbdds_rpy20
jupyter notebook
```

Debería abrirse un navegador con acceso a Jupyter desde donde podrás acceder a los notebooks. Habitualmente el servidor Jupyter se abre en http://localhost:8888/.

## 9.3. Importar datos de Star Wars SWAPI

Datos procesados a partir de https://swapi.dev/documentation#root:

In [1]:
import requests
import pandas as pd
import numpy as np


In [2]:
# exec(open('get_entity.py').read())
def get_entity(entity = None, url_base = 'http://swapi.dev/api/'):
# https://swapi.dev/documentation#root

    page_num = 1
    results_acum = []
    entity_list = ["films","people","planets","species","starships","vehicles"]

    if(entity not in entity_list):
        page_num = None
        print('Call a valid entity: ' + ','.join(entity_list))

    while(page_num is not None):
        url = "".join([url_base, entity, '/?page=', str(page_num)])
        rjson = requests.get(url).json()
        results_acum = results_acum + rjson['results']
        if(rjson['next'] is not None):
            page_num = page_num + 1
        else:
            page_num = None

    if (len(results_acum) > 0):
        results_df = pd.DataFrame(results_acum)
    else:
        results_df = entity_list

    return(results_df)

def str2num(df,num_list):
    num_list = [lab for lab in num_list if lab in df.columns]
    num_list = [lab for lab in num_list if  df[lab].dtype==object]
    for x in num_list:
        if (x=="crew"):
            df.loc[df[x] == "30-165",x] = "165"
        df[x] = pd.to_numeric(df[x].replace('[^0-9\\.-]', '', regex=True), errors='coerce')
    return(df)

In [4]:
res = get_entity()

Call a valid entity: films,people,planets,species,starships,vehicles


In [5]:
res

['films', 'people', 'planets', 'species', 'starships', 'vehicles']

In [8]:
films_df = get_entity("films")
people_df = get_entity("people")
planets_df = get_entity("planets")
species_df = get_entity("species")
starships_df = get_entity("starships")
vehicles_df = get_entity("vehicles")

people_df.index = people_df.name
planets_df.index = planets_df.name
species_df.index = species_df.name
starships_df.index = starships_df.name
vehicles_df.index = vehicles_df.name

people_df.drop("name",inplace=True, axis=1)
planets_df.drop("name",inplace=True, axis=1)
species_df.drop("name",inplace=True, axis=1)
starships_df.drop("name",inplace=True, axis=1)
vehicles_df.drop("name",inplace=True, axis=1)

In [9]:
people_num = ["height","mass","birth_year"]
planets_num = ["diameter","rotation_period","orbital_period","population","surface_water", "gravity"]
species_num = ["average_height","average_lifespan"]
starships_num = ["passengers","length","crew","cost_in_credits","cargo_capacity"
                 ,"hyperdrive_rating","MGLT","max_atmosphering_speed"]
vehicles_num = ["cargo_capacity","cost_in_credits","crew","length","max_atmosphering_speed","passengers"]


In [10]:
people_df.loc[(people_df["gender"].isin(['n/a', 'none'])) | people_df["gender"].isna(),"gender"] = 'none'
planets_df["temperate_tropical"] = planets_df.climate.apply(lambda x: ("temperate" in x) or ("tropical" in x)).astype(int)


In [11]:
people_df2 = str2num(people_df,people_num)
planets_df2 = str2num(planets_df,planets_num)
species_df2 = str2num(species_df,species_num)
starships_df2 = str2num(starships_df,starships_num)
vehicles_df2 = str2num(vehicles_df,vehicles_num)


In [12]:
planets_df2.loc["Bespin","gravity"] = 1.5

In [15]:
people_df2.to_pickle("www/people_df.pkl")
planets_df2.to_pickle("www/planets_df.pkl")
species_df2.to_pickle("www/species_df.pkl")
starships_df2.to_pickle("www/starships_df.pkl")
vehicles_df2.to_pickle("www/vehicles_df.pkl")

In [16]:
vehicles_df2.describe()

cost_in_credits      length  max_atmosphering_speed        crew  \
count        21.000000   38.000000               38.000000   39.000000   
mean      70008.333333   20.466842             1012.131579    8.179487   
std       99279.640830   39.306859             3191.612032   23.512451   
min        2500.000000    1.500000               30.000000    0.000000   
25%        8000.000000    3.885000              100.000000    1.000000   
50%       15000.000000    9.550000              380.000000    1.000000   
75%       75000.000000   16.825000              815.000000    4.000000   
max      350000.000000  210.000000            20000.000000  140.000000   

       passengers  cargo_capacity  
count   38.000000    3.100000e+01  
mean    36.973684    1.316797e+05  
std    102.298843    4.734958e+05  
min      0.000000    0.000000e+00  
25%      0.000000    1.000000e+01  
50%      1.000000    8.000000e+01  
75%     13.500000    5.800000e+03  
max    500.000000    2.000000e+06